In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import logging

import requests
from tqdm import tqdm_notebook
tqdm = tqdm_notebook

import numpy as np
import pandas as pd

from dvidutils import LabelMapper
from libdvid import DVIDNodeService

from neuclease.dvid import *

In [3]:
handler = logging.StreamHandler(sys.stdout)
root_logger = logging.getLogger()
root_logger.handlers = []
root_logger.addHandler(handler)
root_logger.setLevel(logging.INFO)
logging.getLogger('kafka').setLevel(logging.WARNING)

In [4]:
pwd

'/nrs/flyem/bergs/complete-ffn-agglo'

In [6]:
%%time
synapses_s0 = load_synapses_from_csv('sampled-synapses-d585.csv').to_records(index=False)
synapses_s1 = load_synapses_from_csv('sampled-synapses-d585-SCALE-1.csv').to_records(index=False)

CPU times: user 49.1 s, sys: 9.39 s, total: 58.5 s
Wall time: 46 s


In [ ]:
# Convert to rounded scale-0
synapses_s0['z'] = synapses_s0['z'] // 2 * 2
synapses_s0['y'] = synapses_s0['y'] // 2 * 2
synapses_s0['x'] = synapses_s0['x'] // 2 * 2

synapses_s1['z'] *= 2
synapses_s1['y'] *= 2
synapses_s1['x'] *= 2

In [19]:
%%time
synapses_s0.sort()
synapses_s1.sort()

In [22]:
print((synapses_s0['z'] != synapses_s1['z']).sum())
print((synapses_s0['y'] != synapses_s1['y']).sum())
print((synapses_s0['x'] != synapses_s1['x']).sum())

0
0
0


In [23]:
(synapses_s0['label'] != synapses_s1['label']).sum()

48661

In [24]:
48661/71369749

0.0006818154845969824

In [11]:
synapses_s0.shape, synapses_s1.shape

((71369749,), (71369749,))

In [21]:
synapses_s0[:10]

rec.array([(1734, 20468, 23908, 109025850),
           (1734, 20488, 23878, 109025846),
           (1748, 20452, 23888, 109025847),
           (1748, 20458, 23902, 109025862),
           (1748, 20476, 23892, 109025890),
           (1748, 20486, 23908, 109025843),
           (1762, 21516, 23810, 110049280),
           (1768, 21478, 23808, 110049177),
           (1774, 20166, 23892, 109025920), (1776, 21608, 23694, 110049302)], 
          dtype=[('z', '<i4'), ('y', '<i4'), ('x', '<i4'), ('label', '<u8')])

In [20]:
synapses_s1[:10]

rec.array([(1734, 20468, 23908, 109025850),
           (1734, 20488, 23878, 109025846),
           (1748, 20452, 23888, 109025847),
           (1748, 20458, 23902, 109025862),
           (1748, 20476, 23892, 109025890),
           (1748, 20486, 23908, 109025843),
           (1762, 21516, 23810, 110049280),
           (1768, 21478, 23808, 110049177),
           (1774, 20166, 23892, 109025920), (1776, 21608, 23694, 110049302)], 
          dtype=[('z', '<i4'), ('y', '<i4'), ('x', '<i4'), ('label', '<u8')])

In [25]:
%time synapse_table = load_synapses_from_csv('synapses-d585.csv')

CPU times: user 40.7 s, sys: 5.41 s, total: 46.1 s
Wall time: 42.2 s


In [26]:
synapse_table[['z', 'y', 'x']] = synapse_table[['z', 'y', 'x']] // 2 * 2

In [27]:
synapse_table.sort_values(['z', 'y', 'x'], inplace=True)

In [28]:
differences_table = synapse_table.iloc[(synapses_s0['label'] != synapses_s1['label'])]

In [29]:
differences_table.shape

(48661, 6)

In [30]:
differences_table.columns

Index(['x', 'y', 'z', 'kind', 'conf', 'user'], dtype='object')

In [31]:
differences_table['kind'].value_counts()

PreSyn     48453
PostSyn      208
Name: kind, dtype: int64

In [33]:
synapses_s0.shape

(71369749,)

Index(['z', 'y', 'x', 'label'], dtype='object')

In [39]:
%time synapse_table.sort_values(['z', 'y', 'x'], inplace=True)

CPU times: user 50.6 s, sys: 21.6 s, total: 1min 12s
Wall time: 56.4 s


In [40]:
synapse_table['label'] = synapses_s0['label']

In [41]:
test_synapses = synapse_table.iloc[:1000]

CPU times: user 1min 37s, sys: 50.7 s, total: 2min 28s
Wall time: 1min 46s


In [47]:
synapse_counts.iloc[:20]

kind,PostSyn,PreSyn
label,,
106979579,3265.0,20.0
108632992,268.0,24.0
109025843,1.0,NaN
109025846,1.0,NaN
109025847,1.0,NaN
109025850,1.0,NaN
109025857,NaN,1.0
109025862,1.0,NaN
109025881,1.0,NaN


In [48]:
len(synapse_counts)

37972998

1492207

In [ ]:
master_node = ('emdata3:8900', '7254')
mapping = fetch_complete_mappings(*master_node, 'segmentation')

In [55]:
mapper = LabelMapper(mapping.index.values, mapping.values)

In [58]:
synapse_table['body'] = mapper.apply(synapse_table['label'].values, True)

In [60]:
%time synapse_counts = synapse_table[['body', 'kind']].pivot_table(index='body', columns='kind', aggfunc='size')

CPU times: user 1min 32s, sys: 1min 1s, total: 2min 34s
Wall time: 1min 31s


In [61]:
focused_bodies = synapse_counts.query('PostSyn >= 10 or PreSyn > 1')

In [62]:
len(focused_bodies)

544514

In [64]:
#focused_bodies

In [66]:
focused_bodies.fillna(0.0, inplace=True)

/groups/flyem/proj/cluster/miniforge/envs/flyem/lib/python3.6/site-packages/pandas/core/frame.py:3790: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


In [68]:
focused_bodies.to_csv('/nrs/flyem/bergs/complete-ffn-agglo/bodies-0.5-2-tbars-10-psds-7254.csv', index=True, header=True)

In [70]:
!wc /nrs/flyem/bergs/complete-ffn-agglo/bodies-0.5-2-tbars-10-psds-7254.csv

  544515   544515 10746373 /nrs/flyem/bergs/complete-ffn-agglo/bodies-0.5-2-tbars-10-psds-7254.csv
